In [45]:
import duckdb
import pandas as pd

In [44]:
# Conectando e criando o banco .db diretamente (salvo em disco)
con = duckdb.connect("/content/drive/MyDrive/github_projects/data_modeling/tabelas_sql/desnormalizada/tabela_desnormalizada.db")

# Criar a tabela desnormalizada (usando o mesmo código anterior)
con.execute("""
CREATE TABLE desnormalizada AS
SELECT
    o.order_id,
    c.customer_id,
    c.customer_city,
    o.order_status,
    oi.product_id,
    p.product_category_name,
    cat.product_category_name_english,
    oi.price,
    s.seller_city,
    pay.payment_type,
    r.review_score
FROM '/content/drive/MyDrive/github_projects/data_modeling/dataset/olist_orders_dataset.csv' o
JOIN '/content/drive/MyDrive/github_projects/data_modeling/dataset/olist_customers_dataset.csv' c ON o.customer_id = c.customer_id
JOIN '/content/drive/MyDrive/github_projects/data_modeling/dataset/olist_order_items_dataset.csv' oi ON o.order_id = oi.order_id
JOIN '/content/drive/MyDrive/github_projects/data_modeling/dataset/olist_products_dataset.csv' p ON oi.product_id = p.product_id
LEFT JOIN '/content/drive/MyDrive/github_projects/data_modeling/dataset/product_category_name_translation.csv' cat ON p.product_category_name = cat.product_category_name
LEFT JOIN '/content/drive/MyDrive/github_projects/data_modeling/dataset/olist_sellers_dataset.csv' s ON oi.seller_id = s.seller_id
LEFT JOIN '/content/drive/MyDrive/github_projects/data_modeling/dataset/olist_order_payments_dataset.csv' pay ON o.order_id = pay.order_id
LEFT JOIN '/content/drive/MyDrive/github_projects/data_modeling/dataset/olist_order_reviews_dataset.csv' r ON o.order_id = r.order_id;
""")

print("Banco salvo com sucesso no caminho especificado!")


Banco salvo com sucesso no caminho especificado!


In [46]:
con.close()

In [47]:
# Conectar ao banco .db (DuckDB)
con = duckdb.connect("/content/drive/MyDrive/github_projects/data_modeling/tabelas_sql/desnormalizada/tabela_desnormalizada.db")

# Ler tabela com DuckDB, retorna um DataFrame do pandas automaticamente
df = con.execute("SELECT * FROM desnormalizada").df()


In [49]:
display(df)

,order_id,customer_id,customer_city,order_status,product_id,product_category_name,product_category_name_english,price,seller_city,payment_type,review_score
0,8426a61c687c38a2fb35080413ec9eb4,c3b836aea9b06d84b1b9d38b41f137e1,sao paulo,delivered,110adb18eeba9a0984478ee3c9302e16,automotivo,auto,179.49,osasco,credit_card,5
1,84275d513f92f95d3e62865feeaab895,3248720a7617d01317ab868c2bb3676a,curitiba,delivered,2028bf1b01cafb2d2b1901fca4083222,perfumaria,perfumery,56.99,santo andre,credit_card,5
2,842785266c32e96b693fa76cf473a88f,fd06144f27bb7cd0a11db676d656a9c6,sao paulo,delivered,3308e7337d30a51ac18fa4e9e72b310c,telefonia,telephony,27.99,sao paulo,credit_card,4
3,8428cdf79e64b95486e06ad5d0573e22,3303404d891a25079e25394503ae31f6,rio de janeiro,delivered,b61c2827d3a8043826c9132d88188744,esporte_lazer,sports_leisure,369.00,cotia,credit_card,4
4,84295d74f40adb034c954659df22fded,7c2df1f7ce2d2e1cac99169abaca6b31,ananindeua,delivered,e4f2856ddacde505a7d8a1faa4bf2c7d,informatica_acessorios,computers_accessories,58.99,sao paulo,voucher,5
...,...,...,...,...,...,...,...,...,...,...,...
118305,417f9f65f61787a7bec245c5e8c589dc,f2220e827c9230ec561dab611317a232,angra dos reis,delivered,930363fe8355b51e7bb4154214246561,None,None,19.70,sao paulo,voucher,<NA>
118306,5535824c4716c782eed6edd2624a79ce,581ed420cd31ed56bec49cff51d2e42e,uberlandia,delivered,7e28b562fde0b95bd515cde17eb9bbb6,None,None,58.00,atibaia,credit_card,<NA>
118307,03343a88c4d669e6378c5aedc4147919,d6851a9cf4328debeddd2f63212ff9e2,sao paulo,delivered,b36f3c918c91478c4559160022d3f14e,None,None,150.00,santana de parnaiba,credit_card,<NA>
118308,6cfa5243a13a7ea9c5b37fba140d4f9d,c9ea27accd209c13378af27611158cd9,cuiaba,delivered,5a848e4ab52fd5445cdc07aab1c40e48,None,None,122.99,guarulhos,credit_card,<NA>
